In [9]:
import syft as sy
import numpy as np
from syft.core.adp.tensor import Tensor
from syft.core.adp.entity import Entity
from syft.core.adp.adversarial_accountant import AdversarialAccountant

entities = [Entity(name="Tudor"), Entity(name="Madhava"), Entity(name="Kritika"), Entity(name="George")]

x = Tensor(np.array([[1,1],[1,0],[0,1],[0,0]])).private(min_val=0, max_val=1, entities=entities, is_discrete=True)
y = Tensor(np.array([[1],[1],[0],[0]])).private(min_val=0, max_val=1, entities=entities, is_discrete=False)

x = x[0:3]
y = y[0:3]

_weights = Tensor(np.random.uniform(size=(2,1)))

In [10]:
%%timeit -n2 -r2
weights = _weights + 0
acc = AdversarialAccountant(max_budget=3000000)
for i in range(1):
    batch_loss = 0
#     for row in range(len(x)):
# [row:row+1]
    pred = x.dot(weights)
    loss = np.mean(np.square(y-pred))
    loss.backward()

    weight_grad = (weights.grad * 1)
    weight_grad = weight_grad.publish(acc=acc, sigma=1)

    weights = weights - weight_grad
    batch_loss += loss.value

acc.print_ledger()

# Efficiency Ideas:
# - group polynomial terms by entity (eliminates the need to search)
# - parallelize publishing tensor variables in tensor (but only if variables come from separate entities)
# X offer discretization + brute force search
# - replace variables by uniqueneses from left to right in polynomial for memoization
# - for search algorithms on iterative learning functions - have it start searching at the global optima from the last iteration
# - faster polynomial evaluation
# - offer partial discretization

Tudor	1.473721812941998
Kritika	1.2626747203310007
Madhava	0.7005916285533693
Tudor	1.473721812941998
Kritika	1.2626747203310007
Madhava	0.7005916285533693
Tudor	1.473721812941998
Kritika	1.2626747203310007
Madhava	0.7005916285533693
Tudor	1.473721812941998
Kritika	1.2626747203310007
Madhava	0.7005916285533693
4min 51s ± 3.64 s per loop (mean ± std. dev. of 2 runs, 2 loops each)


Tudor	7.122362375325958
Kritika	5.842118932733819
Madhava	3.555589348429929
Tudor	7.122362375325958
Kritika	5.842118932733819
Madhava	3.555589348429929
Tudor	7.122362375325958
Kritika	5.842118932733819
Madhava	3.555589348429929
Tudor	7.122362375325958
Kritika	5.842118932733819
Madhava	3.555589348429929
4.58 s ± 132 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


In [71]:
weights


Tensor([[0.6556921576076219],
        [0.7788766774053854]], dtype=object)